# 🐍 Projeto Rápido: Analisador de Dados CSV (Google Colab)

Este notebook roda **100% no Colab** (sem precisar instalar nada no seu PC).

### Como usar
1. **Execute as células na ordem** (Ctrl+Enter ou o botão ▶️ à esquerda).
2. Na etapa de *Upload*, envie um arquivo chamado **`dados_vendas.csv`** com as colunas:
   - `data` (YYYY-MM-DD), `produto`, `categoria`, `quantidade`, `preco_unitario`
3. Se você **não enviar** um CSV, o notebook **gera um exemplo automático** para você treinar.
4. No final, baixe os arquivos gerados: `grafico_receita_por_mes.png` e `resumo_metricas.txt`.

> Dica: Depois, suba o gráfico e o resumo no seu **GitHub** com um README bonitinho. 😉

In [ ]:
# 1) Instalar bibliotecas necessárias (rápido)
!pip -q install pandas matplotlib

In [ ]:
# 2) Upload opcional do CSV (ou geração automática de exemplo)
from google.colab import files
import io, numpy as np, pandas as pd
from datetime import datetime

print("\n➡️ Se você tiver um arquivo 'dados_vendas.csv', faça o upload agora.")
print("   Caso contrário, vou gerar um exemplo automático para você.")
uploaded = files.upload()

if 'dados_vendas.csv' in uploaded:
    df = pd.read_csv(io.BytesIO(uploaded['dados_vendas.csv']), parse_dates=['data'])
    print("\n✅ CSV carregado com sucesso! Linhas:", len(df))
else:
    rng = np.random.default_rng(42)
    products = [
        ("Caderno", "Papelaria", 18.90),
        ("Caneta", "Papelaria", 4.50),
        ("Mochila", "Acessórios", 129.90),
        ("Lapiseira", "Papelaria", 12.00),
        ("Agenda", "Papelaria", 32.00),
        ("Marca-texto", "Papelaria", 7.90),
        ("Estojo", "Acessórios", 39.90),
    ]
    date_range = pd.date_range("2025-01-01", "2025-08-31", freq="D")
    rows = []
    for _ in range(120):
        d = rng.choice(date_range)
        prod, cat, base_price = products[rng.integers(0, len(products))]
        q = int(rng.integers(1, 12))
        price = round(float(base_price * (1 + rng.normal(0, 0.10))), 2)
        price = max(0.5, price)
        rows.append((d, prod, cat, q, price))
    df = pd.DataFrame(rows, columns=["data", "produto", "categoria", "quantidade", "preco_unitario"])
    print("\nℹ️ Nenhum CSV enviado. Gerado um exemplo automático com", len(df), "linhas.")

# garantir tipos e coluna de receita
df['data'] = pd.to_datetime(df['data'])
df['quantidade'] = pd.to_numeric(df['quantidade'], errors='coerce').fillna(0)
df['preco_unitario'] = pd.to_numeric(df['preco_unitario'], errors='coerce').fillna(0.0)
df['receita'] = df['quantidade'] * df['preco_unitario']

print("\nPrévia do dataframe:")
display(df.head())

In [ ]:
# 3) Métricas principais e agregações
import pandas as pd
registros = len(df)
itens_total = int(df['quantidade'].sum())
receita_total = float(df['receita'].sum())
ticket_medio_por_venda = float(df['receita'].mean())

top_prod = (
    df.groupby('produto')['receita']
      .sum()
      .sort_values(ascending=False)
      .head(5)
      .reset_index()
)

receita_mes = (
    df.set_index('data')
      .resample('M')['receita']
      .sum()
      .reset_index()
)

print("Métricas principais:")
print({
    'registros': registros,
    'itens_total': itens_total,
    'receita_total': round(receita_total, 2),
    'ticket_medio_por_venda': round(ticket_medio_por_venda, 2)
})

print("\nTop 5 produtos por receita:")
display(top_prod)

print("\nReceita por mês:")
display(receita_mes)

In [ ]:
# 4) Gráfico simples (matplotlib, 1 gráfico, sem estilos/cores específicas)
import matplotlib.pyplot as plt
plt.figure()
plt.bar(receita_mes['data'].dt.strftime('%Y-%m'), receita_mes['receita'])
plt.title('Receita por mês')
plt.xlabel('Mês')
plt.ylabel('Receita (R$)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('grafico_receita_por_mes.png', dpi=150)
plt.show()
print('✅ Gráfico salvo como grafico_receita_por_mes.png')

In [ ]:
# 5) Salvar resumo das métricas em texto e disponibilizar para download
resumo = []
resumo.append(f"Registros: {registros}")
resumo.append(f"Itens vendidos (total): {itens_total}")
resumo.append(f"Receita total: R${receita_total:.2f}")
resumo.append(f"Ticket médio por venda: R${ticket_medio_por_venda:.2f}")
resumo.append("")
resumo.append("Top 5 produtos por receita:")
resumo.append(top_prod.to_string(index=False))
resumo.append("")
resumo.append("Receita por mês:")
resumo.append(receita_mes.to_string(index=False))

with open('resumo_metricas.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(resumo))

print('✅ Arquivo resumo_metricas.txt gerado.')

In [ ]:
# 6) Downloads (rode esta célula para baixar os arquivos gerados)
from google.colab import files
files.download('grafico_receita_por_mes.png')
files.download('resumo_metricas.txt')